### 1. Implement Graphene-pair encoding tokernizer

### 2. Load dataset

g:\4th year\1st sem\Research Project\experiments\gpe-reproduce\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['idx', 'src', 'tgt'],
    num_rows: 10000
})
{'idx': 0, 'src': 'Some 14 months later, the second calf is born.', 'tgt': 'சுமார் 14 மாதங்கள் கழித்து, இரண்டாம் கன்றை ஈனுகிறது.'}


### 3. Implement calculating compression ratio function

### 4. function for train Graphene-pair encoding tokernizer

### 5. function for train Byte-pair encoding tokernizer

### 6. Train Graphene-pair encoding tokernizer

Training on 8000 tamil texts

Sample training text: சுமார் 14 மாதங்கள் கழித்து, இரண்டாம் கன்றை ஈனுகிறது.
Graphemes: ['சு', 'மா', 'ர்', ' ', '1', '4', ' ', 'மா', 'த', 'ங்', 'க', 'ள்', ' ', 'க', 'ழி', 'த்', 'து', ',', ' ', 'இ', 'ர', 'ண்', 'டா', 'ம்', ' ', 'க', 'ன்', 'றை', ' ', 'ஈ', 'னு', 'கி', 'ற', 'து', '.']
Number of graphemes: 35

Training tokenizer with vocab_size=1000...
Starting grapheme-based BPE training...
Initial vocabulary size: 188
Sample graphemes: ['ச', 'ு', 'ம', 'ா', 'ர', '்', '<', '/', 'w', '>', ' ', '1', '4', 'த', 'ங', 'க', 'ள', 'ழ', 'ி', ',']
Merge 0: ('ம்', '</w>') -> ம்</w> (freq: 6861)
Merge 100: ('0', '</w>') -> 0</w> (freq: 368)
Merge 200: ('செய்', 'ய') -> செய்ய (freq: 209)
Merge 300: ('வி', 'ரு') -> விரு (freq: 140)
Merge 400: ('யெ', 'கோவா') -> யெகோவா (freq: 111)
Merge 500: ('க்', 'கூ') -> க்கூ (freq: 89)
Merge 600: ('ப்பை', '</w>') -> ப்பை</w> (freq: 75)
Merge 700: ('ளா', 'ல்</w>') -> ளால்</w> (freq: 62)
Merge 800: ('வ', 'ங்கி') -> வங்கி (freq: 55)
Final vocabula

### 7. Train Byte-pair encoding tokernizer


HuggingFace BPE tokenizer saved.
🔹 HF BPE Compression Ratio: 2.9452891717743204


### 8. Overview of comparission


📊 Compression Ratio Comparison
GPE Tokenizer:      2.92
HuggingFace BPE:    2.95


Training on 997 sinhala texts

Sample training text: සඳුදා දින, ස්ටැන්ෆර්ඩ් සරසවි වෛද්‍ය පාසලේ විද්‍යාඥයෝ, සෛලයක ආකාරය අනුව එය වර්ග කළ හැකි නව දෝෂ නිර්ණය කිරීමේ මෙවලමක්: එනම්, එකක් ඇමරිකානු ඩොලර් ශතයක පමණ මුදලකින් නිෂ්පාදනය කළ හැකි සාමාන්‍ය ඉන්ක්ජෙට් මුද්‍රණ යන්ත්‍රයකින් මුද්‍රණය කළ හැකි ඉතා කුඩා චිපයක් සොයාගත් බවට නිවේදනය කළහ.
Graphemes: ['ස', 'ඳු', 'දා', ' ', 'දි', 'න', ',', ' ', 'ස්', 'ටැ', 'න්', 'ෆ', 'ර්', 'ඩ්', ' ', 'ස', 'ර', 'ස', 'වි', ' ', 'වෛ', 'ද්\u200d', 'ය', ' ', 'පා', 'ස', 'ලේ', ' ', 'වි', 'ද්\u200d', 'යා', 'ඥ', 'යෝ', ',', ' ', 'සෛ', 'ල', 'ය', 'ක', ' ', 'ආ', 'කා', 'ර', 'ය', ' ', 'අ', 'නු', 'ව', ' ', 'එ', 'ය', ' ', 'ව', 'ර්', 'ග', ' ', 'ක', 'ළ', ' ', 'හැ', 'කි', ' ', 'න', 'ව', ' ', 'දෝ', 'ෂ', ' ', 'නි', 'ර්', 'ණ', 'ය', ' ', 'කි', 'රී', 'මේ', ' ', 'මෙ', 'ව', 'ල', 'ම', 'ක්', ':', ' ', 'එ', 'න', 'ම්', ',', ' ', 'එ', 'ක', 'ක්', ' ', 'ඇ', 'ම', 'රි', 'කා', 'නු', ' ', 'ඩො', 'ල', 'ර්', ' ', 'ශ', 'ත', 'ය', 'ක', ' ', 'ප', 'ම', 'ණ', ' ', 'මු', 'ද', 'ල', 'කි', 'න්', ' ', 'නි', 'ෂ්', 'පා


HuggingFace BPE tokenizer saved.
🔹 HF BPE Compression Ratio: 2.6817242578612492



📊 Compression Ratio Comparison
GPE Tokenizer:      2.44
HuggingFace BPE:    2.68
